 <div style="text-align:center;">
  <span style="color:green; font-size:2em; font-weight:bold;">Modélisation</span><br><br>

# <span style="color:green">Introduction</span><br><br> 

### <span style="color:green">Références et équations de base de la relation d'Okun</span><br><br> 

Dans cette partie, nous allons proposer une modélisation permettant de vérifier empiriquement la loi d'Okun, à partir des bases de données établies dans la partie "Préparation_données.ipynb".

La loi d'Okun, indique que : "chaque fois que le PIB chute d'un certain pourcentage, par rapport au produit potentiel, le taux de chômage augmente d'environ un point de pourcentage. Ainsi, lorsque le PIB réel diminue, le taux de chômage augmente." (Mankiw, 2003)

Pour cette modélisation, on s'inspire des travaux suivants : 

- [1]. Traoré, D. L., Diakite, S., & Mariko, O. (2021). Croissance et chômage au Mali : Vérification empirique de la loi d'Okun. Revue Malienne de Science et de Technologie, Série C : Sciences Humaines et Sociales, 02(25), Page. ISSN 1987-1031. CNRST, Bamako, Mali.

- [2]. Aassif, Z. L'extraction de la tendance cycle. Haut-Commissariat au Plan, Direction de la comptabilité nationale. 

- [3]. Belaidi, N. (2022, 28 mars). Validation croisée en Machine Learning. Consulté le  12 décembre 2023. URL : https://blent.ai/blog/a/validation-croisee-machine-learning.


Le travail [1] nous présente deux équations de base de la relation d'Okun, la version en "différences premières" et la "version gap".

- Version en "différences premières" : 
$$\Delta u = C + \beta\Delta \text{pib} + \epsilon$$

- "Version gap" : 
$$u - u^* = c + \beta^*(\text{pib} - \text{pib}^*) + \epsilon$$

avec $u$ le taux de chômage, $u^*$ le taux de chômage naturel, $\text{pib}^*$ le PIB potentiel et c, $\beta$ et $\epsilon$  des constantes à déterminer grâce à la modélisation. 

### <span style="color:green">Equation et outils utilisés dans notre modélisation</span><br><br> 

Pour la modélisation, nous allons utiliser la version "gap", qui relie le taux de chômage et le PIB de la manière suivante : 

$$u - u^* = c + \beta^*(\text{pib} - \text{pib}^*) + \epsilon$$

Pour extraire la tendance à long terme du taux de chômage et du PIB (c'est-à-dire, le taux de chômage naturel et le PIB potentiel), à partir de la base de données, nous allons utiliser le filtre de Hodrick-Prescott. Ce filtre est introduit dans [2], et permet de décomposer une série selon sa tendance à long terme et son cycle (fluctuations autour de la tendance) à plus court terme.

Par ailleurs, pour prendre en compte la dynamique temporelle de nos données, nous allons utiliser le modèle ARDL (Auto Regressive Distributed Lag), introduit dans [2].

### <span style="color:green">Les différentes étapes de notre modélisation</span><br><br> 


Pour effectuer notre modélisation sur les données obtenues suite au traitement, nous allons tout d'abord regrouper les pays présents selon différents critères, grâce à la technique du clustering. 
Nous sélectionnerons un pays de manière aléatoire au sein de chaque groupe afin de vérifier la loi d'Okun (ou alors nous vérifierons la loi d'Okun pour chaque groupe après agrégation).

(Par ailleurs, nous allons utiliser une méthode de validation croisée afin de valider notre modèle. Nous allons tout d'abord établir les paramètres du modèle grâce au jeu d'entrainement reprenant les données de 1994 à 2016 ; puis nous vérifierons la précision du modèle grâce au jeu de test reprenant les données de 2017 à 2023.)


# <span style="color:green"> I- Extraction de la tendance à long terme du taux de chômage et du PIB</span><br><br> 

## <span style="color:green"> Importation des bases de données</span><br><br> 

In [ ]:
import declarations as d 

In [ ]:
# Bases de données relatives au PIB et au taux de chômage
quart_data = d.pd.read_csv('bases/quart_data.csv', encoding='latin-1')
data_moyenne = d.pd.read_csv('bases/donnees_moyennees.csv', encoding = 'latin-1')

In [ ]:
# Base de données relative aux variables considérées pour le clustering

## <span style="color:green">Extraction de la tendance à long terme pour le PIB (gdp)</span><br><br> 

In [ ]:
quart_data.head(2)

## <span style="color:green">Extraction de la tendance à long terme pour le taux de chômage (rate)</span><br><br> 

In [ ]:
# Extraire toutes les colonnes avec le suffixe "_rate" ainsi que la colonne des dates
columns_rate = ['Dates'] + [col for col in quart_data.columns if col.endswith('_rate')] 
df_rates = quart_data[columns_rate]
df_rates.set_index('Dates', inplace = True)

df_rates.head(2)

In [ ]:
# Extraire toutes les colonnes avec le suffixe "_gdp" ainsi que la colonne des dates
columns_gdp = ['Dates'] + [col for col in quart_data.columns if col.endswith('_gdp')] 
df_gdp = quart_data[columns_gdp]
df_gdp.set_index('Dates', inplace = True)

df_gdp.head(2)

In [ ]:
import statsmodels.tsa.filters.hp_filter as smf
import statsmodels.tsa.ardl as sma
import statsmodels.api as sm

 ## <span style="color:green">Exemple de la Belgique</span><br><br> 

### <span style="color:green">Extraction de la tendance de la série</span><br><br> 

In [ ]:
# Extraction de la tendance long terme du taux de chômage
x = df_rates['BEL_rate']

# Appliquer le filtre de Hodrick-Prescott
tendance_x, cycle_x = smf.hpfilter(x, lamb=1600)

# Afficher les résultats
d.plt.plot(x, label='Série originale')
d.plt.plot(tendance_x, label='Tendance')
d.plt.plot(cycle_x, label='Cycle')
d.plt.legend()
d.plt.show()

In [ ]:
# Extraction de la tendance long terme du PIB
y = df_gdp['BEL_gdp']

# Appliquer le filtre de Hodrick-Prescott
tendance_y, cycle_y = smf.hpfilter(y, lamb=1600)

# Afficher les résultats
d.plt.plot(x, label='Série originale')
d.plt.plot(tendance_y, label='Tendance')
d.plt.plot(cycle_y, label='Cycle')
d.plt.legend()
d.plt.show()

In [ ]:
nouveau_x = x - tendance_x
nouveau_y = y - tendance_y

In [ ]:
dfx = nouveau_x.reset_index().rename(columns={'index': 'Dates'})
dfx = dfx.rename(columns = {0:'rates'})

In [ ]:
dfy = nouveau_y.reset_index().rename(columns={'index': 'Dates'})
dfy = dfy.rename(columns = {0 :'gdp'})

In [ ]:
data_Belgique = d.pd.merge(dfx, dfy, on='Dates', how='inner')
data_Belgique.head(2)

### <span style="color:darkred">Régression linéaire simple avec la méthode OLS</span><br><br> 

La régression linéaire simple considère chaque observation comme indépendante des autres, ce qui signifie qu'elle ne modélise pas l'autocorrélation ou la dépendance temporelle potentielle entre les observations successives. 

In [ ]:
# S'assurer que la colonne Dates est au format datetime
data_Belgique['Dates'] = d.pd.to_datetime(data_Belgique['Dates'])

# Ajouter une constante à vos données pour inclure l'intercept dans le modèle
data_Belgique['const'] = 1

# Spécifier les variables indépendantes (X) et dépendante (y)
X = data_Belgique[['const', 'rates']]
y = data_Belgique['gdp']

# Créer et ajuster le modèle de régression linéaire
model = sm.OLS(y, X)
results = model.fit()

# Afficher les résultats du modèle
print(results.summary())


### <span style="color:darkred">Modèle ARIMA (AutoRegressive Integrated Moving Average)</span><br><br> 

Ici, on considère un modèle de séries temporelles, qui capture la dépendance temporelle et modélise la dynamique temporelle des données.

In [ ]:
# AACF (AutoCorrelation Function)
# PACF (Partial AutoCorrelation Function)

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Tracez l'ACF et PACF
fig, ax = d.plt.subplots(1, 2, figsize=(12, 4))
plot_acf(data_Belgique['gdp'], lags=20, ax=ax[0])
plot_pacf(data_Belgique['gdp'], lags=20, ax=ax[1])
d.plt.show()


In [ ]:
# Créer et ajuster le modèle ARIMA
model = sm.tsa.ARIMA(data_Belgique['gdp'], order=(2, 2, 11))  # Spécifier l'ordre du modèle ARIMA
results = model.fit()

# Afficher les résultats du modèle
print(results.summary())


### <span style="color:darkred">Modèle ARDL</span><br><br> 

In [ ]:
# Spécifiez les paramètres du modèle ARDL
endog_variable = data_Belgique['gdp']
lags_endog = 2  # Nombre de retards pour la variable endogène
exog_variables = data_Belgique['rates']
lags_exog = [1, 2]  # Nombre de retards pour les variables exogènes
trend = 'c'  # 'c' pour une constante, 'ct' pour une constante et une tendance linéaire, ou None pour aucun

# Créez le modèle ARDL
model_ardl = sma.ARDL(endog_variable, lags=lags_endog, exog=exog_variables, order=lags_exog, trend=trend)

# Ajustez le modèle
results = model_ardl.fit()

# Affichez les résultats du modèle
print(results.summary())


In [ ]:
# Spécifiez les retards que vous souhaitez inclure
lags = 2

# Créez des colonnes de lags pour rates
for i in range(1, lags + 1):
    data_Belgique[f'rates_lag{i}'] = data_Belgique['rates'].shift(i)

# Spécifiez les variables indépendantes (X) et dépendante (y)
X = data_Belgique[['const', 'rates_lag1', 'rates_lag2']]
y = data_Belgique['gdp']

# Éliminez les lignes avec des valeurs manquantes introduites par les lags
data_Belgique = data_Belgique.dropna()

# Créez et ajustez le modèle de régression linéaire avec lags
model = sm.OLS(y, X)
results = model.fit()

# Affichez les résultats du modèle
print(results.summary())


# II- Clustering : regroupement des pays selon des critères de développement

# III- Vérification de la loi pour un pays tiré de manière i.i.d. dans chaque groupe
